<a href="https://colab.research.google.com/github/monocas/Machine-Learning----Queimadas-Cerrado/blob/main/Projeto_ML_Oficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **RISCO DE FOGO: QUEIMADAS NO CERRADO**
<hr>
<p align="justify">
O primeiro bloco da disciplina de Aprendizado de Máquina trata da coleta e análise exploratória de dados. Desenvolvemos um modelo de previsão para incidência de focos de incêndio no Cerrado a partir do banco de dados de queimadas do INPE. O grupo é composto pelas alunas:
<p align="justify">
Isabela Bento Beneti </p>
<p align="justify">
Monyque Karoline de Paula Silva </p>
<p align="justify">
Sofia Baccega C.C. de Oliveira </p>
<p align="justify">
Sophia Figueiredo Michel </p>
<hr>

### **Início:**
<p align="justify">
Neste bloco tratamos e analisamos os dados obtidos a respeito dos focos de incêndio registrados no Cerrado durante o período Janeiro-Julho de 2022. Para isso, construímos e filtramos o dataframe e utilizamos bibliotecas de processamento, análise e plotagem de dados.
</p>

### **Coleta:**

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from itertools import product
from matplotlib import pyplot as plt
from sklearn import tree

In [7]:
#Dados do Cerrado Brasileiro, armazenados mensalmente via .csv
arquivos = ['janeiro.csv','fev.csv','marco.csv','abril.csv','maio.csv','junho.csv','julho.csv']

In [8]:
def read_csv(list_of_files):
    dfs = []
    for x in list_of_files:
        file = pd.read_csv(x, sep=',')
        dfs.append(file)
    df = pd.concat(dfs).reset_index(drop=True)
        
    return df

In [12]:
meses = read_csv(arquivos)

Utilizamos o comando drop, da biblioteca pandas, para eliminar as linhas dos dados não atribuídos (NaN).

In [13]:
#Dados trabalhados para a eliminação de NaN nas colunas e identificação dos tipos de dados
meses.convert_dtypes()
meses.dropna(how='any', inplace=True)

In [14]:
## Para rodar o código pela primeira vez, descomente essas linhas

meses.drop(columns=['Unnamed: 0','estado', 'municipio', 'pais', 'municipio_id', 'estado_id', 'pais_id', 'bioma'], axis = 1, inplace=True)

In [15]:
meses['satelite']= meses['data_hora_gmt']
meses['satelite'] = meses['satelite'].str[11:13] #criando coluna "hora" a partir de "data_hora_gmt"
meses['data_hora_gmt'] = meses['data_hora_gmt'].str[6] #criando coluna "mes" a partir de "data_hora_gmt"
meses_fogo = meses['risco_fogo'] > 0
meses.rename(columns={'satelite': 'hora'}, inplace= True)
meses.rename(columns={'data_hora_gmt': 'mes'}, inplace = True)
meses = meses[meses_fogo]
meses.describe

<bound method NDFrame.describe of               lat        lon mes hora  numero_dias_sem_chuva  precipitacao  \
775    -12.314859 -43.234066   3   01                    9.0           2.7   
778    -20.227880 -46.415940   3   01                    8.0           0.0   
779    -20.229250 -46.426820   3   01                    8.0           0.0   
780    -20.238930 -46.425510   3   01                    8.0           0.0   
781    -20.237550 -46.414600   3   01                    8.0           0.0   
...           ...        ...  ..  ...                    ...           ...   
287431  -6.360000 -45.670000   7   23                   46.0           0.0   
287432  -6.360000 -45.690000   7   23                   46.0           0.0   
287433  -6.400000 -44.140000   7   23                   34.0           0.0   
287434  -6.350000 -45.670000   7   23                   46.0           0.0   
287435  -6.340000 -45.690000   7   23                   45.0           0.0   

        risco_fogo  
775     

Após a seleção de colunas, identificamos os tipos de dados utilizados em cada coluna. Para o nosso dataframe, quisemos verificar quais variáveis numéricas temos e quais teremos que converter para float.

In [16]:
meses.dtypes, meses.shape

(lat                      float64
 lon                      float64
 mes                       object
 hora                      object
 numero_dias_sem_chuva    float64
 precipitacao             float64
 risco_fogo               float64
 dtype: object, (265141, 7))

Então, alteramos a coluna "data_hora_gmt" de string (object) para float:

In [17]:
meses["mes"] = meses["mes"].astype(float)
meses["hora"] = meses["hora"].astype(float)
print(meses.dtypes)

lat                      float64
lon                      float64
mes                      float64
hora                     float64
numero_dias_sem_chuva    float64
precipitacao             float64
risco_fogo               float64
dtype: object


In [18]:
display(meses.describe())

,lat,lon,mes,hora,numero_dias_sem_chuva,precipitacao,risco_fogo
count,265141.000000,265141.000000,265141.000000,265141.000000,265141.000000,265141.000000,265141.000000
mean,-11.978888,-48.191533,5.959584,13.805519,21.485572,0.098769,0.850110
std,4.324839,4.013527,1.203238,6.113204,60.515029,0.611814,0.255347
min,-24.617660,-60.029380,2.000000,0.000000,-999.000000,0.000000,0.010000
25%,-14.334340,-50.130000,5.000000,12.000000,7.000000,0.000000,0.800000
50%,-11.840000,-47.163000,6.000000,16.000000,20.000000,0.000000,1.000000
75%,-8.780950,-45.208540,7.000000,17.000000,37.000000,0.000000,1.000000
max,-2.383860,-41.698070,7.000000,23.000000,129.000000,21.050000,1.000000


In [19]:
meses = meses.reset_index()
del meses['index']
display(meses)

,lat,lon,mes,hora,numero_dias_sem_chuva,precipitacao,risco_fogo
0,-12.314859,-43.234066,3.0,1.0,9.0,2.7,1.0
1,-20.227880,-46.415940,3.0,1.0,8.0,0.0,0.3
2,-20.229250,-46.426820,3.0,1.0,8.0,0.0,0.3
3,-20.238930,-46.425510,3.0,1.0,8.0,0.0,0.4
4,-20.237550,-46.414600,3.0,1.0,8.0,0.0,0.4
...,...,...,...,...,...,...,...
265136,-6.360000,-45.670000,7.0,23.0,46.0,0.0,1.0
265137,-6.360000,-45.690000,7.0,23.0,46.0,0.0,1.0
265138,-6.400000,-44.140000,7.0,23.0,34.0,0.0,1.0
265139,-6.350000,-45.670000,7.0,23.0,46.0,0.0,1.0


Após a identificação dos tipos de dados existentes no código, normalizamos os dados por meio do comando "zscore". Esse processo de normalização é extremamente importante para análises estátisticas por transformar a escala da distribuição de uma variável e tornar coerente as proporções entre conjunto de elementos.

(abaixo, em comentário, temos uma demonstração da normalização manual)

In [20]:
#c_maximo = meses['numero_dias_sem_chuva'].max()
#c_minimo = meses['numero_dias_sem_chuva'].min()

#p_maximo = meses['precipitacao'].max()
#p_minimo = meses['precipitacao'].min()

#lat_maximo = meses['lat'].max()
#lat_minimo = meses['lat'].min()

#lon_maximo = meses['lon'].max()
#lon_minimo = meses['lon'].min()

#meses['numero_dias_sem_chuva'] = (meses['numero_dias_sem_chuva'] - c_minimo) / (c_maximo - c_minimo)
#meses['precipitacao'] = (meses['precipitacao'] - p_minimo) / (p_maximo - p_minimo)
#meses['lat'] = (meses['lat'] - lat_minimo) / (lat_maximo - lat_minimo)
#meses['lon'] = (meses['lon'] - lon_minimo) / (lon_maximo - lon_minimo)


#display(meses)